# Adaptive CMA-ES configurations - Analysis

This Python Notebook covers the analysis of pre-processed data for the adaptive CMA-ES research.

The input data consists of CSVs with the required runtimes for each pre-specified target for all runs, separated into files for each function/dimensionality pair. For details on the pre-processing that went into creating this data, please refer to the `pre-processing.ipynb/html` notebook.

An example with target values 100, 1 and 0.01:

`idx |   1   2   3 ...   56   57   58   59 ...    164    165    166    167`<br>
`val | 124 102  94 ... 1.13 1.06 0.98 0.96 ... 0.0123 0.0109 0.0098 0.0097`

For section `$\inf$-100`, the first index where the value is _below_ 100 is 3. For the next target the index is 58, and for the final target the first index is 166.


> Sander van Rijn<br>
> s.j.van.rijn@liacs.leidenuniv.nl<br>
> LIACS<br>
> 2018-03-29

## Initialization

In [ ]:
%matplotlib inline

from __future__ import division, print_function

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
from collections import Counter, defaultdict
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# Some utility functions for dealing with the representations

# First, some hardcoded variables
num_options_per_module = [2]*9        # Binary part
num_options_per_module.extend([3]*2)  # Ternary part
max_length = 11
factors = [2304, 1152, 576, 288, 144, 72, 36, 18, 9, 3, 1]

def list_all_representations():
    """ Create a list of all possible representations for the modular CMA-ES.
        Each representation is itself a list with <max_length> integer entries {0, 1, ..., n},
        where 'n' is the number of options for the module in that position.
    """
    products = []
    # count how often there is a choice of x options
    counts = Counter(num_options_per_module)
    for num, count in sorted(counts.items(), key=lambda x: x[0]):
        products.append(product(range(num), repeat=count))
    all_representations = []
    for representation in list(product(*products)):
        all_representations.append(list(sum(representation, ())))
    return all_representations


def reprToString(representation):
    """ Function that converts the structure parameters of a given ES-structure representation to a string

        >>> reprToInt([0,0,0,0,0,1,0,1,0,1,0])
        >>> '00000101010'
    """
    return ''.join([str(i) for i in representation[:max_length]])

In [ ]:
data_location = '/media/rijnsjvan/Data/SurfDrive/Research Data/Adaptive ES/anytime_convergence/data/'
# file_name = 'steepness_data_{ndim}D-f{fid}.csv'
# file_name = 'interpolated_ART_data_{ndim}D-f{fid}.csv'
# file_name = 'stepwise_ERT_data_{ndim}D-f{fid}.csv'
file_name = 'ERT_data_{ndim}D-f{fid}.csv'


#TODO: Retrieve this information from the files instead?
ndims = [5, 20]
fids = [1, 10, 15, 20]

num_steps = 51
powers = np.round(np.linspace(2, -8, num_steps), decimals=1)
target_values = np.power([10]*num_steps, powers)

columns = ['10e{}'.format(power) for power in powers]


np.set_printoptions(linewidth=1000, edgeitems=30)
pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 2000)

# Exploration
Using the data in (one of) the CSV files, we'll start with some exploratory analysis.
Any useful procedures will be coded as functions, making them easy to re-use for a final full analysis later on.

In [ ]:
def get_data(ndim, fid):
    return pd.read_csv(data_location + file_name.format(ndim=ndim, fid=fid), index_col=0)

# Starting with 5D F1 as an example
df = get_data(ndim=5, fid=1)
df.head()

In [ ]:
def absolutetodifferences(df):
    return_df = df.copy()
    df_columns = list(reversed(columns))
    for column_a, column_b in zip(columns, columns[1:]):
        return_df[column_b] = df[column_b] - df[column_a]
    return return_df

# Next, we get the differences between the running times for each of the targets
df_diff = absolutetodifferences(df)
df_diff.head()

We can already see a problem here: Some of the smallest values are zeroes.
This means that the actual improvement step made by the algorithm went across the specified targets.

In [ ]:
print(df_diff[columns].min(axis=0))

In [ ]:
def aggregateByMean(df):
    df_means = df.groupby(by=['Representation', 'ndim', 'function ID']).mean(numeric_only=True)
    df_means = df_means.drop(columns=['instance ID', 'repetition'])
    df_means = df_means.reset_index()
    return df_means

def aggregateByStd(df):
    df_std = df.groupby(by=['Representation', 'ndim', 'function ID']).std()
    df_std = df_std.drop(columns=['instance ID', 'repetition'])
    df_std = df_std.reset_index()
    return df_std

def aggregateByMax(df):
    df_means = df.groupby(by=['Representation', 'ndim', 'function ID']).max(axis=1)
    df_means = df_means.drop(columns=['instance ID', 'repetition'])
    df_means = df_means.reset_index()
    return df_means

def aggregateByMedian(df):
    df_means = df.groupby(by=['Representation', 'ndim', 'function ID']).median(numeric_only=True)
    df_means = df_means.drop(columns=['instance ID', 'repetition'])
    df_means = df_means.reset_index()
    return df_means

def aggregateBy(df, aggregation):
    if aggregation == 'mean':
        return aggregateByMean(df)
    elif aggregation == 'std':
        return aggregateByStd(df)
    elif aggregation == 'max':
        return aggregateByMax(df)
    elif aggregation == 'median':
        return aggregateByMedian(df)
    else:
        raise ValueError("Invalid choice for 'aggregation': {aggregation}")

In [ ]:
# # Then we can aggregate
# df_diff = aggregateByMax(absolutetodifferences(df))
# df_diff.head()

If we aggregate before calculating the differences, we can end up disrupting the monotonically decreasing nature of our data. This would result in a negative difference score!

Instead, by first calculating the differences, we can then aggregate over the actually existing differences

In [ ]:
section_labels = ['\inf - 10e{}'.format(powers[0])]
section_labels.extend(['10e{} - 10e{}'.format(a, b) for a, b in zip(powers[:-1], powers[1:])])

def plotOnAxis(df, columns, ax, *, title=None, legend=True):
    for idx, row in df.iterrows():
        ax.plot(row[columns].values, '-', label=row['Representation'])

    ax.set_ylabel("\'steepness\'")
    ax.set_xlabel('convergence sections')
    ax.set_xticks(np.arange(len(target_values)))
    ax.set_xticklabels(section_labels)
    ax.xaxis.set_tick_params(rotation=90)
    if title:
        ax.set_title(title)
    if legend:
        ax.legend(loc=0)

In [ ]:
# Let's plot the head (=first 5 rows) of the currently loaded data as an example
fig, ax = plt.subplots(figsize=(12,8))
plotOnAxis(df_diff.head(), columns, ax)
plt.show()

In [ ]:
# Extracting only the configurations that are best between a pair of targets rather than all 4,608...
def selectSmallestPerColumn(df, columns, nsmallest=1):
    indices = set()
    for col in columns:
        if df[col].max() > 0:
            new_indices = set(df[col].nsmallest(nsmallest).axes[0])
            indices = indices.union(new_indices)
    return df.iloc[sorted(indices)]

selectSmallestPerColumn(df_diff, columns).head()

In [ ]:
# Now we plot all differences for each configuration that is best between at least one pair of targets
subset = selectSmallestPerColumn(df_diff, columns)
fig, ax = plt.subplots(figsize=(12,8))
plotOnAxis(subset, columns, ax, legend=False)
plt.show()

We get some really large values here, which are actually not what we want to see: higher values mean bad performance.

Instead, at least to visualize, it's more interesting to look at the 1/$\delta$'s.

In [ ]:
def makecleaninverses(df, columns):
    return_df = df.copy()
    for col in columns:
        return_df[col] = 1/df[col]
    return_df = return_df.replace(np.inf, np.NaN)
    return return_df

In [ ]:
# Now we plot all differences for each configuration that is best between at least one pair of targets
subset = makecleaninverses(selectSmallestPerColumn(df_diff, columns), columns)
fig, ax = plt.subplots(figsize=(12,8))
plotOnAxis(subset, columns, ax, legend=False)
plt.yscale('log')
plt.show()

In [ ]:
#SOME DEBUGGING INFO
# idx = df_diff[columns[14]].nsmallest(1).axes[0]
# print(idx)
# rep = df_diff.iloc[idx]['Representation'].values[0]
# print(rep)

# subset = df[df['Representation'] == rep]

# print(idx)
# print(subset.as_matrix(columns).astype(int))
# print(df_diff.iloc[idx].as_matrix(columns))

#SOME DEBUGGING INFO
# print(absolutetodifferences(aggregateByMean(subset)).as_matrix(columns).astype(int))

# Analysis of all experiments

In [ ]:
# Just some caching
full_data = {(ndim, fid): get_data(ndim=ndim, fid=fid) for ndim, fid in product(ndims, fids)}

In [ ]:
# Because it's defined as a simple-to-use function, we can now plot this data for all available datasets
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(12,8), sharex=True, sharey=True)

for exp, ax in zip(product(ndims, fids), axes.flatten()):
    ndim, fid = exp
#     df_means = aggregateByMean(absolutetodifferences(full_data[(ndim, fid)]))
    df_means = absolutetodifferences(full_data[(ndim, fid)])
    subset = makecleaninverses(selectSmallestPerColumn(df_means, columns), columns)
    plotOnAxis(subset, columns, ax, title='{}D F{}'.format(ndim, fid), legend=False)
    ax.set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
df_diff = absolutetodifferences(get_data(ndim=5, fid=1))
df_diff[columns] = df_diff[columns].replace(0, np.NaN)  # we replace 0 by NaN so they're automatically excluded
# df_diff = aggregateByMax(df_diff)
subset = selectSmallestPerColumn(df_diff, columns)

# And now the hypothetical result if we take the best sub-result per section...
print(subset[columns].min(axis=0))
print(subset[columns].min(axis=0).sum())

In [ ]:
def findlastreachedcolumn(df, columns):
    actual_best = np.NaN
    col_idx = 0
    while np.isnan(actual_best):
        col_idx += 1
        actual_best = df[columns[-col_idx]].min()

    return columns[-col_idx]


def calcresults(data):
    records = []
    labels = ['ndim', 'fid', 'Target', 'Empirical best', 'Adaptive best', 'Relative improvement']
    for ndim, fid in product(ndims, fids):
        df = data[(ndim, fid)]

#         df_means = aggregateByMean(df)
        last_column = findlastreachedcolumn(df, columns)
        actual_best = df[last_column].min()

        df_diff = absolutetodifferences(df)
        df_diff[columns] = df_diff[columns].replace(0, np.NaN)
#         df_diff = aggregateByMax(df_diff)
        subset = selectSmallestPerColumn(df_diff, columns)
        theory_best = subset[columns].min(axis=0).sum()

        rel_improvement = 1-(theory_best/actual_best)

        records.append((ndim, fid, last_column, actual_best, theory_best, rel_improvement))

    results = pd.DataFrame.from_records(records, columns=labels)
    return results

print(calcresults(full_data))

In [ ]:
file_name = 'stepwise_ERT_data_{ndim}D-f{fid}.csv'  # Using different files
old_full_data = full_data
full_data = {(ndim, fid): get_data(ndim=ndim, fid=fid) for ndim, fid in product(ndims, fids)}

print(calcresults(full_data))

file_name = 'ERT_data_{ndim}D-f{fid}.csv'  # resetting the variable...
full_data = old_full_data

# Alternative approach: Limited splits

So far we've looked at finding the optimal performance when you can pick the best convergence in between each pair of targets. Practically, this is not really feasible as you would need to make sure when exactly you are switching, which is very difficult in a black-box setting.

But, as we've seen that there are (significant) gains to be made, so let's scale it down. Instead of allowing arbitrary switching, what happens if we allow only a few switches?

In [ ]:
def determinebestsplit(df, columns, *, numsplits=1, aggregation='mean'):
    best_split_idx = 0
    best_value = np.inf
    final_column_idx = columns.index(findlastreachedcolumn(df, columns))

    for split in range(1,final_column_idx):
        
        part1 = df[df.columns[3:]]
        part1 = part1.assign(values=(df[columns[split]]))

        part2 = df[df.columns[3:]]
        part2 = part2.assign(values=(df[columns[final_column_idx]] - df[columns[split]]))

        part1_idx = part1['values'].idxmin()
        val1 = part1.iloc[part1_idx]['values']
        x = part1.iloc[part1_idx]

        part2_idx = part2['values'].idxmin()
        val2 = part2.iloc[part2_idx]['values']
        x = part2.iloc[part2_idx]

        if val1+val2 < best_value:
            best_value = val1+val2
            best_split_idx = split
            best_idx_1, best_idx_2 = part1_idx, part2_idx
    
    return best_value, best_split_idx, best_idx_1, best_idx_2


def calculatesplitbasedrecord(df, aggregation='mean'):
    value, split_idx, part1_idx, part2_idx = determinebestsplit(df, columns, aggregation=aggregation)

    last_column = findlastreachedcolumn(df, columns)
    actual_best_idx = df[last_column].idxmin()
    actual_best_value = df.iloc[actual_best_idx][last_column]
    
    return (actual_best_value, value, 1-(value/actual_best_value), 
            columns[split_idx], actual_best_idx, part1_idx, part2_idx)


def calculatesplitbasedoverview(data, cases, *, aggregation='mean'):
    records = []
    for ndim, fid in cases:
        df = data[(ndim, fid)]
        record = calculatesplitbasedrecord(df, aggregation=aggregation)
        records.append((ndim, fid, *record))

    labels = ['ndim', 'fid', 'Static', 'Adaptive', 'relative improvement', 
              'split', 'Static index', '$C_1$ index', '$C_{\\Gamma}$ index']
    results = pd.DataFrame.from_records(records, columns=labels)
    return results

In [ ]:
# mean_split_overview = calculatesplitbasedoverview(product(ndims, fids))
# print(mean_split_overview)
split_overview = calculatesplitbasedoverview(full_data, product(ndims, fids))
print(split_overview)

In [ ]:
def displayoverview(overview, data):
    representations = defaultdict(list)
    cols = ['Static index', '$C_1$ index', '$C_{\\Gamma}$ index']
    for idx, row in overview.iterrows():
        df = data[(row.loc['ndim'], row.loc['fid'])]
        for column in cols:
            representations[column[:-6]+' rep'].append(df.iloc[row.loc[column]]['Representation'].replace(' ', ''))
        
    disp_overview = overview.assign(**representations)
    disp_overview = disp_overview.drop(columns=cols)
    
    return disp_overview
    
displayoverview(split_overview, full_data)

## Convergence plots

What follows are a some plots of convergence behavior:
 * 1) the best original configuration
 * 2) the (entire) configuration that performs best **before** the split (**part1**)
 * 3) the (entire) configuration that performs best **after** the split (**part2**)
 * 4) the **compound** performance of **part1** before the split and **part2** after the split
 
NOTE:
The plots currently only give an indication of what we would like to see. Until the vertical line (i.e., the split location), the compound line follows the convergence of ***part1***, and follows the behavior of ***part2*** after the split. 

For now, the results are not what we would expect, as the red line (compound) should always result in reaching the furthest target with the lowest hitting time. The reason for this, is that these results are (for now) plotted using the mean-aggregated hitting time data, which has the problem that monotonicity is not maintained, while the **calculations** are done on the differences. This still has to be changed.

In [ ]:
def plotCompoundOnAxis(df, row, ax, *, title='', log=False):
    for column in ['Static index', '$C_1$ index', '$C_{\\Gamma}$ index']:
        orig_idx = int(row.loc[column])
        ax.plot(df.iloc[orig_idx][columns].values, '-', label=column[:-6])

    split_idx = columns.index(row.loc['split'])
    idx1 = row.loc['$C_1$ index']
    part1 = df.iloc[idx1][columns[:split_idx+1]].values

    idx2 = row.loc[r"$C_{\Gamma}$ index"]
    part2 = df.iloc[idx2][columns[split_idx:]].values
    part2 = (part2 - part2[0]) + df.iloc[idx1][columns[split_idx]]
    compound = np.concatenate((part1, part2[1:]))
    ax.plot(compound, '-', label='Adaptive')

    num_valids = len(compound[~np.isnan(compound.astype(np.float))])
    
    ax.axvline(split_idx, linewidth=1, color='black')

    ax.set_ylabel("AHT")
    ax.set_xlabel(r"$\phi$")
    ax.set_xticks(np.arange(len(target_values))[::5])
    ax.set_xticklabels(target_values[::5])
    ax.set_xlim([0, num_valids])
    ax.xaxis.set_tick_params(rotation=90)
    ax.set_title(title)
    ax.legend(loc=0)
    if log:
        ax.set_yscale('log')
    
    

def plotCompounds(overview, data, cases, fig, *, log=defaultdict(lambda: False), save_name=None):    
    
    for idx, row in overview.iterrows():

        exp, ax = cases[idx]
        ndim, fid = exp
        title = '{ndim}D F{fid}'.format(ndim=ndim, fid=fid)
        df = data[(ndim, fid)]

        plotCompoundOnAxis(df, row, ax, title=title, log=log[(ndim, fid)])

    fig.tight_layout()
    if save_name:
        fig.savefig(save_name+'.pdf')
        fig.savefig(save_name+'.png')

log = defaultdict(lambda: True, {(5, 1): False, (5, 10): False})
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(12,8))
cases = list(zip(product(ndims, fids), axes.flatten()))

plotCompounds(split_overview, full_data, cases, fig, log=log, save_name='interpolated_ERT_splits')
fig.show()

The results for 5D F1 and F10 are the clearest and most promising. Especially 5D F10 shows how `part 2` has a clear knee-point that is optimally identified and suggests a potential improvement of over 16%.

On the other hand, the results for 20D F20 seem not very informative as most target values are not reached.

The remaining functions are dominated by ERT scores that are near the maximum, i.e., only a few runs succeeded up to the point where the split is identified. After that, the behavior is no longer determined by the `max_budget` penalty for every failed run, but only the behavior of those runs that did succeed is taken into account in determining these results. This makes the approach a greedy one, that is fine with taking the risk that several runs will not succeed, simply because after a while, it can ignore the penalty given for that. Note that this is an inherent problem of aggregating in any way over a set of runs which did not all reach the desired target value, regardless of whether it is measured by ERT or bootstrapped aRT.

An alternative explanation might be that these are configurations that require a lot of time to learn internal parameters before 'getting in the flow' that allows them to make progress in the optimization task. (B)IPOP with its adaptive population size is an example of a module that could exhibit such behavior. If that is the case, it would be interesting to examine the parameter values when the knee-point occurs. At the same time, it would have to be taken into account when implementing this in an actual adaptive configuration setting.

## Leaving out (B)IPOP variants

To examine the potential influence of the second possible explanation in the previous section, we now repeat the experiment with all (B)IPOP variants excluded from the dataset.

In [ ]:
def filterbipop(df):
    df = df.assign(bipop=df['Representation'].astype(str).str[28] != '0')
    df = df[df['bipop'] == False]
    df = df.drop(columns=['bipop'])
    return df.reset_index(drop=True)

filtered_data = {(ndim, fid): filterbipop(full_data[(ndim, fid)]) for ndim, fid in product(ndims, fids)}

In [ ]:
filtered_split_overview = calculatesplitbasedoverview(filtered_data, product(ndims, fids))
displayoverview(filtered_split_overview, filtered_data)

In [ ]:
log = defaultdict(lambda: True, {(5, 1): False, (5, 10): False})
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(12,8))
cases = list(zip(product(ndims, fids), axes.flatten()))

plotCompounds(filtered_split_overview, filtered_data, cases, fig, log=log, save_name='no-bipop_interpolated_ERT_splits')

Overall, the results seem the same: Only 5D F1 and F10 can really be plotted without having to resort to a log-scale. The most noticable thing is that the best 'regular' results are now worse than before, as the succesful (B)IPOP module is no longer allowed in the configurations.

# Only fully successful

As we're still dealing with the fitness 'jumps' caused by the ERT calculation, we'll now look only at those configurations which were 100% successful. This does mean we will reach fewer targets, but those results will not be distorted and are therefore more reliable.

Note: (for now) this is only a one-time filter based on the smallest convergence target. Especially for the single-split variant, it may still be interesting to perform this filter for every possible 'part1' by itself.

In [ ]:
file_name = 'interpolated_ART_data_{ndim}D-f{fid}.csv'

raw_data = {(ndim, fid): get_data(ndim=ndim, fid=fid) for ndim, fid in product(ndims, fids)}

In [ ]:
def aggregateByFilteredERT(df, max_budget):
    df = df.drop(columns=['instance ID', 'repetition'])

    sums = df.fillna(max_budget).groupby(by=['Representation', 'ndim', 'function ID']).sum()
    counts = df.groupby(by=['Representation', 'ndim', 'function ID']).count()
    
    # Count how many runs should be successful and discard anything below that value
    max_successful = counts.max()[0]
    counts[counts != max_successful] = np.NaN
    
    ERTs = sums/counts
    ERTs = ERTs.replace(np.inf, np.NaN).reset_index()
    return ERTs

In [ ]:
# Some caching again
successful_only_data = {(ndim, fid): aggregateByFilteredERT(raw_data[(ndim, fid)], max_budget=ndim*10e4) for ndim, fid in product(ndims, fids)}

In [ ]:
calcresults(successful_only_data)

In [ ]:
successful_split_overview = calculatesplitbasedoverview(successful_only_data, product(ndims, fids))
displayoverview(successful_split_overview, successful_only_data)

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(12,8))
cases = list(zip(product(ndims, fids), axes.flatten()))

plotCompounds(successful_split_overview, successful_only_data, cases, fig,
              save_name='successful_only_interpolated_ERT_splits')

In [ ]:
experiments = product(ndims, fids[:2])
selection = successful_split_overview.iloc[[0,1,4,5]].reset_index()

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(9,6))
cases = list(zip(experiments, axes.flatten()))

plotCompounds(selection, successful_only_data, cases, fig, 
              save_name='successful_only_interpolated_ERT_splits_p1')

fig.show()

In [ ]:
experiments = product(ndims, fids[2:])
selection = successful_split_overview.iloc[[2,3,6,7]].reset_index()

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(9,6))
cases = list(zip(experiments, axes.flatten()))

plotCompounds(selection, successful_only_data, cases, fig, 
              save_name='successful_only_interpolated_ERT_splits_p2')

fig.show()